<img src="https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png" style="float: left; margin: 10px;">

# 1.3 - Intro to Web Scraping Lab

---

Week 7- 1.3

### LEARNING OBJECTIVES
*In this lab you will learn how to*
- How to read and understand the code in a Scrapy Spider class
- Write a spider to scrape job post from Indeed.com

The following class IndeedSpider is incomplete. It is missing code for two functions: **fill_url_list** and **build_indeed_url**. These functions build and populate a list with every unqiue combination of job search URLs based on the job query key terms provided below. 

### Direction
1. Build those functions and scrape Indeed.com 
2. Annotate every line of code with a short description on how what that line of code is doing

## NOTE: 

Once you're finished proto-type these two functions in this notebook, copy the code over to the scrapy files but **DO NOT SCRAPE ALL 9 Choose 3 PAGES**. If everyone does this, GA will be blocked from visiting Indeed.com by the site's servers. Instead, only run 2 or 3 URLS to test and verify that your code works. 



In [4]:
# -*- coding: utf-8 -*-

import scrapy
from webscraper.items import WebscraperItem

class IndeedSpider(scrapy.Spider):
    name = "indeed_spider"
    allowed_domains = ["indeed.com"]


    # job query key terms 
    job_sites = ["https://www.indeed.com/jobs"]
    job_titles = ["Data Scientist", 'Machine Learning Engineer', "Data Analyst"]
    cities_states = [("San Francisco", "CA"), ("Los Angeles", "CA"), ("San Diego", "CA")]

    # increment page in order to scrap additonal posting in same city
    page_increment_list = ['0','1','2']

    # list populated with unique combinations of job query key terms
    #these combinations form unique urls to scrap 
    #start_urls = self.build_indeed_url(job_sites, job_titles, cities_states)
    

    start_urls = (
        'https://www.indeed.com/jobs?q=Data+Analyst&l=San+Diego%2C+CA&start=0',
        'https://www.indeed.com/jobs?q=Data+Analyst&l=San+Diego%2C+CA&start=10',
        'https://www.indeed.com/jobs?q=Data+Analyst&l=San+Diego%2C+CA&start=20')

    def fill_url_list(self, job_sites, job_titles, cities_states):
        '''Return a list with every combination of urls for job searchs. 
        Each combination mush as job_site, job_titles, and cities_states in the following format:

        EXAMPLE: https://www.indeed.com/jobs?q=Data+Scientist&l=San+Bernardino%2C+CA&start=2''

        # you will need to call build_url inside this function '''
        return build_indeed_url(self, job_sites, job_titles, cities_states, page_increment_list)


    def build_indeed_url(self, job_sites, job_title, cities_states, incremental_page = ['0']):
        '''Build site url template with specifc arguments.
           Return a correct url address that can be used to request an Indeed page. 
           The url should be able to work on the chrome browser'''
        url_list = []
        i = job_sites[0] + "?q="
        for j in job_title:
            j = j.replace(" ", "+")
            j = j + "&l="
            for k in cities_states:
                k = str(k)
                k = k.replace(", ", "%2C+")
                k = k.replace(" ", "+")
                k = k.replace(")", "")
                k = k.replace("(", "")
                k = k.replace("'", "")
                for l in incremental_page:
                    l = int(l)*10
                    page = "&start={}".format(l)
                    url_list.append(i+j+k+page)
        return url_list


    def parse(self, response):
        titles = response.xpath('//h2/a/@title').extract()
        links = response.xpath('//h2/a/@href').extract()
        companies = response.xpath('//span[@class="company"]/span/text()').extract()

        for title, link, comp in zip(titles, links, companies):
            item = WebscraperItem()
            item['title'] = title
            abs_url = response.urljoin(link)
            item['url'] = abs_url
            item['company'] = comp

            request = scrapy.Request(abs_url, callback=self.parse_job)
            request.meta['item'] = item
            yield request


    def parse_job(self, response):
        item = response.meta['item']
        keys = ['sql', 'python', 'ml', 'spark', 'hadoop', 'dl',
                'stats', 'ts', 'de', 'dp', 
                'nlp', 'bd', 'pca', 'pred_modeling', 
                'anomaly_detection', 'data_analysis', 
                'exp_design', 'cnn', 'bayes_opt', 'pipeline']

        skills = ['sql', 'python', 'machine learning',
                  'spark', 'hadoop', 'deep learning', 
                  'statistics', 'time series', 'data engineering', 
                  'data products', 'nlp', 'big data', 
                  'pca', 'predictive modeling', 'anomaly detection', 
                  'data analysis', 'experimental design', 
                  'deep convolutional networks','bayesian optimization',
                  'pipeline']

        for key, skill in zip(keys, skills):
            item[key] = int(skill in response.text.lower())

        yield item


## Test and build your functions below

Since there are 9 choose 3 possible combinations for URL based on the job query key terms provided, you know that you're functions works when: 

1. The URLs that your functions output work on your browser's search bar. 
2. There are 84 URLs outputed by the fill_url_list function. 

In [110]:
# job query key terms 
job_sites = ["https://www.indeed.com/jobs"]
job_titles = ["Data Scientist", 'Machine Learning Engineer', "Data Analyst"]
cities_states = [("New York", 'NY'), ("Los Angeles", "CA"), ("San Diego", 'CA')]

# increment page in order to scrap additonal posting in same city
page_increment_list = ['0','1','2']

# list populated with unique combinations of job query key terms
#these combinations form unique urls to scrap  
#start_urls = self.build_indeed_url(job_sites, job_titles, cities, states)


start_urls = ('https://www.indeed.com/jobs?q=Data+Scientist&l=San+Bernardino%2C+CA&start=2')

In [122]:
def build_indeed_url(job_sites, job_title, cities_states, incremental_page = ['0']):
    '''Build site url template with specifc arguments.
       Return a correct url address that can be used to request an Indeed page. 
       The url should be able to work on the chrome browser'''
    url_list = []
    i = job_sites[0] + "?q="
    for j in job_title:
        j = j.replace(" ", "+")
        j = j + "&l="
        for k in cities_states:
            k = str(k)
            k = k.replace(", ", "%2C+")
            k = k.replace(" ", "+")
            k = k.replace(")", "")
            k = k.replace("(", "")
            k = k.replace("'", "")
            for l in incremental_page:
                l = int(l)*10
                page = "&start={}".format(l)
                url_list.append(i+j+k+page)
    return url_list

In [126]:
def fill_url_list(job_sites, job_titles, cities_states):
    '''Return a list with every combination of urls for job searchs.
    Each combination mush as job_site, job_titles, and cities_states in the following format:
    
    EXAMPLE: https://www.indeed.com/jobs?q=Data+Scientist&l=San+Bernardino%2C+CA&start=2''
    
    # you will need to call build_url inside this function '''
    
    return build_indeed_url(job_sites, job_titles, cities_states, page_increment_list)

In [127]:
start_urls = fill_url_list(job_sites, job_titles, cities_states)
start_urls

['https://www.indeed.com/jobs?q=Data+Scientist&l=New+York%2C+NY&start=0',
 'https://www.indeed.com/jobs?q=Data+Scientist&l=New+York%2C+NY&start=10',
 'https://www.indeed.com/jobs?q=Data+Scientist&l=New+York%2C+NY&start=20',
 'https://www.indeed.com/jobs?q=Data+Scientist&l=Los+Angeles%2C+CA&start=0',
 'https://www.indeed.com/jobs?q=Data+Scientist&l=Los+Angeles%2C+CA&start=10',
 'https://www.indeed.com/jobs?q=Data+Scientist&l=Los+Angeles%2C+CA&start=20',
 'https://www.indeed.com/jobs?q=Data+Scientist&l=San+Diego%2C+CA&start=0',
 'https://www.indeed.com/jobs?q=Data+Scientist&l=San+Diego%2C+CA&start=10',
 'https://www.indeed.com/jobs?q=Data+Scientist&l=San+Diego%2C+CA&start=20',
 'https://www.indeed.com/jobs?q=Machine+Learning+Engineer&l=New+York%2C+NY&start=0',
 'https://www.indeed.com/jobs?q=Machine+Learning+Engineer&l=New+York%2C+NY&start=10',
 'https://www.indeed.com/jobs?q=Machine+Learning+Engineer&l=New+York%2C+NY&start=20',
 'https://www.indeed.com/jobs?q=Machine+Learning+Engineer&l